## Dask

Dask - это библиотека для параллельных вычислений и масштабирования. Допустим, что Numpy не справляется с объемом данных, который нам нужно будет обработать. В таком случае естественной альтернативой как раз станет Dask. Dask предоставляет возможность работать с данными, которые превышают объем оперативной памяти, и эффективно использовать ресурсы как на локальной машине, так и на кластере. Библиотека позволяет масштабировать код Python с минимальными изменениями.

## Полезные ссылки

[Тык](https://docs.dask.org/en/stable/)

[Тык](https://tutorial.dask.org/00_overview.html)

[Тык](https://habr.com/ru/companies/otus/articles/759552/)

## Установка

Здесь все стандартно:

In [22]:
!pip install "dask[complete]"

# Dask Array

Dask Array — это масштабируемая версия массива NumPy, которая работает с массивами, превышающими объем оперативной памяти, и распределяет вычисления между несколькими ядрами или машинами.

- Dask Array поддерживает API NumPy, что позволяет использовать знакомые функции и методы.

- Массивы разбиваются на более мелкие блоки, которые обрабатываются независимо.

- Поддерживаются вычисления как на локальной машине, так и в распределенных системах.

Рассмотрим для начала пример из документации:

In [24]:
import numpy as np
import dask.array as da

# Создаем обычный NumPy массив размером 10x10
x = np.arange(100).reshape(10, 10)
# Преобразуем NumPy массив в Dask Array с чанками (разбиением) 5x5
dask_array = da.from_array(x, chunks=(5, 5))

result = dask_array.mean()
print(result.compute())

KeyboardInterrupt: 

Разберемся, что здесь вообще происходит.

Вместо того чтобы обрабатывать весь массив сразу, Dask делит его на чанки, что позволяет работать с большими данными, превышающими оперативную память.
Выполнение вычислений параллелизуется, что ускоряет процесс на многопроцессорных системах.

![dask-array](images/dask-array-2.png)

На изображении показано, как Dask Array разбивает большой массив на более мелкие части — чанки. Каждый чанк представляет собой отдельный NumPy массив.

Пусть нам нужно вычислить индекс растительности NDVI для спутниковых данных, представляющих 10 сцен, каждая из которых состоит из двух каналов: ближний инфракрасный (NIR) и красный (Red). NDVI является важным показателем для анализа растительности, так как он показывает степень активности фотосинтеза в растительных покровах.

In [ ]:
# Генерируем массив данных: снимки 10000x10000 (10 сцен, 2 канала: NIR и Red)
satellite_data = da.random.random((10, 2, 10_000, 10_000), chunks=(1, 2, 5000, 5000))

# Разделяем каналы
nir = satellite_data[:, 0, :, :]  # Near Infrared
red = satellite_data[:, 1, :, :]  # Red

# Вычисляем NDVI
ndvi = (nir - red) / (nir + red)

print("NDVI сцены 1, пиксель [0, 0]:", ndvi[0, 0, 0].compute())

Попробуем сделать то же самое с NumPy (этот код ниже лучше не запускать - у меня, например, все 16 гигов оперативки кончились моментально)

In [ ]:
# Генерируем аналогичный массив данных: снимки 10000x10000 (10 сцен, 2 канала)

# numpy_satellite_data = np.random.random((10, 2, 10_000, 10_000))

# Разделяем каналы
# nir = numpy_satellite_data[:, 0, :, :]  # Near Infrared
# red = numpy_satellite_data[:, 1, :, :]  # Red

# Вычисляем NDVI
# try:
    # ndvi = (nir - red) / (nir + red)
    # print("NDVI сцены 1, пиксель [0, 0]:", ndvi[0, 0, 0])
# except MemoryError:
    # print("NumPy не справился: данные слишком велики для оперативной памяти.")

Таким образом, мы посмотрели как работают массивы в Dask. Теперь потыкаем Bag

In [ ]:
import dask.bag as db
import random
import string
from dask.distributed import Client

client = Client()
print(client.dashboard_link)

# Функция для генерации случайной строки с набором слов
def random_text(num_words=50):
    words = [''.join(random.choices(string.ascii_lowercase, k=random.randint(3, 10))) for _ in range(num_words)]
    return ' '.join(words)

def process_line(line):
    words = line.split()
    return [word.lower() for word in words if word.isalpha()]

num_lines = 50_000
chunk_size = 10_000
data = db.from_sequence((random_text() for _ in range(num_lines)), npartitions=num_lines // chunk_size)

processed_data = data.map(process_line).flatten()  # Обрабатываем строки и объединяем их в один поток

# Подсчитываем частоту каждого слова
word_counts = processed_data.frequencies()

# Получаем топ-10 самых частых слов
top_10_words = word_counts.topk(10, key=lambda x: x[1])
print("Топ-10 самых частых слов:", top_10_words.compute())

## Как проверить прогресс выполнения

Dask поддерживает визуализацию выполнения задач через инструмент Dask Dashboard. Вы можете запустить его, добавив следующие строки:

In [ ]:
from dask.distributed import Client
client = Client()
print(client.dashboard_link)  # Откроет ссылку на дашборд

После запуска вы сможете видеть статус выполнения задач в режиме реального времени.